In [1]:
import ee
import geemap
import geopandas as gpd
import pandas as pd

from pathlib import Path

ee.Initialize()

In [2]:
data = Path("../data/geojson/s1_asc_2019.geojson")
gdf = gpd.read_file(data)


gdf = gdf[["date", "relativeOrbitNumber_start", "platform_number", "system_id", "geometry"]]
gdf.head(3)

,date,relativeOrbitNumber_start,platform_number,system_id,geometry
0,2019-06-21,50,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...,"POLYGON ((-137.83402 60.54038, -138.93672 60.4..."
1,2019-06-21,50,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...,"POLYGON ((-138.45711 60.48614, -137.38779 60.5..."
2,2019-06-21,50,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...,"POLYGON ((-137.54621 63.17687, -137.67419 63.5..."


In [3]:
# isolate the by the platform number and relative orbit number
gdf_50_A = gdf[(gdf["platform_number"] == "A") & (gdf["relativeOrbitNumber_start"] == 50)]
gdf_50_A.head(3)

,date,relativeOrbitNumber_start,platform_number,system_id,geometry
0,2019-06-21,50,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...,"POLYGON ((-137.83402 60.54038, -138.93672 60.4..."
1,2019-06-21,50,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...,"POLYGON ((-138.45711 60.48614, -137.38779 60.5..."
2,2019-06-21,50,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...,"POLYGON ((-137.54621 63.17687, -137.67419 63.5..."


In [4]:
# create a date mosaic

def create_swath_img(gdf: gpd.GeoDataFrame):
    mosaic = None
    for date, group in gdf.groupby("date"):
        if mosaic is None:
            mosaic = ee.ImageCollection(group["system_id"].tolist()).mosaic()
        else:
            mosaic = mosaic.addBands(ee.ImageCollection(group["system_id"].tolist()).mosaic())

    return mosaic.select("V.*")


In [5]:
gdf_79_A = gdf[(gdf["platform_number"] == "A") & (gdf["relativeOrbitNumber_start"] == 79)]
gdf_79_A.head(3)

,date,relativeOrbitNumber_start,platform_number,system_id,geometry
8,2019-06-23,79,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190623T02...,"POLYGON ((-136.83927 59.94516, -136.93962 59.9..."
9,2019-06-23,79,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190623T02...,"POLYGON ((-132.86966 60.31919, -132.51597 60.3..."
10,2019-06-23,79,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190623T02...,"POLYGON ((-137.70888 62.96406, -138.35892 62.8..."


In [6]:
gdf_108_A = gdf[(gdf["platform_number"] == "A") & (gdf["relativeOrbitNumber_start"] == 108) ]
gdf_108_A.head(3)

,date,relativeOrbitNumber_start,platform_number,system_id,geometry
16,2019-06-25,108,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190625T02...,"POLYGON ((-127.92113 58.69370, -127.90423 58.6..."
17,2019-06-25,108,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190625T02...,"POLYGON ((-128.35212 60.18655, -128.35206 60.1..."
18,2019-06-25,108,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190625T02...,"POLYGON ((-130.04902 63.11379, -130.90376 63.0..."


In [7]:
gdf_137_A = gdf[(gdf["platform_number"] == "A") & (gdf["relativeOrbitNumber_start"] == 137)]
gdf_137_A.head(3)

,date,relativeOrbitNumber_start,platform_number,system_id,geometry
24,2019-06-27,137,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190627T02...,"POLYGON ((-123.89251 59.03919, -123.89387 59.0..."
25,2019-06-27,137,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190627T02...,"POLYGON ((-128.81230 60.11465, -128.79740 60.1..."
26,2019-06-27,137,A,COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190627T02...,"POLYGON ((-125.95770 61.93351, -125.25471 61.9..."


In [8]:
swath_50_A = create_swath_img(gdf_50_A)
swath_79_A = create_swath_img(gdf_79_A)
swath_108_A = create_swath_img(gdf_108_A)
swath_137_A = create_swath_img(gdf_137_A)
mosaic = ee.ImageCollection([swath_108_A, swath_137_A]).mosaic()

# Note to self
- When trying to mosic images together, make sure the images are the same size. Otherwise, it wont render anything.
- if the number of images that are in the swath are different than the number of im 

In [9]:
Map = geemap.Map()
Map.addLayer(mosaic, {"min": -25, "max": 10}, "Mosaic")
# Map.addLayer(swath_50_A, {"min": -25, "max": 10}, "Swath 50 A")
# Map.addLayer(swath_79_A, {"min": -25, "max": 10}, "Swath 79 A")
# Map.addLayer(swath_108_A, {"min": -25, "max": 10}, "Swath 108 A")
# Map.addLayer(swath_137_A, {"min": -25, "max": 10}, "Swath 137 A")
# Map.addLayer(swath_123_B, {"min": -25, "max": 10}, "Swath 123 B")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [12]:
rel_orbits = [50, 79, 108]

for orbit_num, group in gdf.groupby(['relativeOrbitNumber_start', 'date']):
    if orbit_num[0] in rel_orbits:
        print(orbit_num)
        print(group)
    

(50, '2019-06-21')
         date  relativeOrbitNumber_start platform_number  \
0  2019-06-21                         50               A   
1  2019-06-21                         50               A   
2  2019-06-21                         50               A   
3  2019-06-21                         50               A   
4  2019-06-21                         50               A   
5  2019-06-21                         50               A   
6  2019-06-21                         50               A   
7  2019-06-21                         50               A   

                                           system_id  \
0  COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...   
1  COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...   
2  COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...   
3  COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...   
4  COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...   
5  COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T02...   
6  COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190621T0